In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import openpyxl
%matplotlib inline
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.1)

In [2]:
# Previo

In [3]:
AÑOS_A_PROCESAR = [str(año) for año in range(2019, 2024)] # ['2019', '2020', '2021', '2022', '2023']

RUTA_BASE_DATOS = 'data'
SUBCARPETAS = ['difusas', 'puntuales', 'ruta'] 
CARPETA_GRAFICOS = f'graficos_{AÑOS_A_PROCESAR[0]}-{AÑOS_A_PROCESAR[-1]}_reporte_es'

NOMBRE_CIUDAD_FILTRO = 'Valdivia'
NOMBRE_MP25 = 'MP2.5'
NOMBRE_MP10 = 'MP10'
TOP_N_CONTAMINANTES_CIUDAD = 20

MAPA_CONTAMINANTES = {
    'NOx ': 'NOx', 'Nitrogen oxides (NOx)': 'NOx',
    'Compuestos Orgánicos Volátiles': 'COV', 'Volatile organic compounds (VOC)': 'COV',
    'Carbon dioxide': 'Dióxido de carbono (CO2)',
    'Carbon monoxide': 'Monóxido de carbono (CO)', 'Monóxido de carbono': 'Monóxido de carbono (CO)',
    'Sulfur dioxide': 'Dióxido de azufre (SO2)', 'Dióxido de azúfre (SO2)': 'Dióxido de azufre (SO2)',
    'Sulfur oxides (SOx)': 'SOx',
    'Arsenic': 'Arsénico',
    'Lead': 'Plomo',
    'Mercury': 'Mercurio',
    'Ammonia': 'Amoniaco (NH3)', 'Nitrógeno amoniacal (o NH3)': 'Amoniaco (NH3)',
    'MP2,5': NOMBRE_MP25, 'PM2.5, primary': NOMBRE_MP25, 'MP2.5': NOMBRE_MP25,
    'PM10, primary': NOMBRE_MP10, 'MP10': NOMBRE_MP10,
    'PM, primary': 'Material Particulado Total', 'Material particulado': 'Material Particulado Total',
    'Toluene': 'Tolueno', 'Tolueno / metil benceno / Toluol / Fenilmetano': 'Tolueno',
    'Benzene': 'Benceno',
    'PCDD-F': 'Dibenzoparadioxinas policloradas y furanos (PCDD/F)',
}

MAPEO_ESTANDARIZACION_REGION = {
    'Metropolitana de Santiago': 'Metropolitana', 'Región Metropolitana de Santiago': 'Metropolitana',
    'Región del Gral. Carlos Ibáñez del Campo': 'Aysén',
    'Aysen del General Carlos Ibanez del Campo': 'Aysén',
    'Aysén del Gral. Carlos Ibañez del Campo': 'Aysén',
    'Aysén del Gral. Carlos Ibáñez del Campo':'Aysén',
    'Libertador Gral. Bernardo O Higgins': "O'Higgins", "O'Higgins": "O'Higgins",
    'Libertador General Bernardo O Higgins': "O'Higgins",
    "Libertador Gral. Bernardo O'Higgins":"O'Higgins",
    'Nuble': 'Ñuble',
    'Magallanes y de la Antártica Chilena':'Magallanes',
    'Los Ríos': 'Los Ríos',
}

In [4]:
# %% [markdown]
# ## Cargar datos

In [5]:
# %%
def cargar_y_preparar_datos(ruta_base, subcarpetas_fuentes, lista_años,
                           mapa_contaminantes_est, mapa_regiones_est):
    """
    Carga, renombra columnas, filtra, estandariza y unifica datos de emisiones.
    Maneja archivos CSV (UTF-8/Latin-1) y Excel (.xlsx, .xls).
    La lógica de renombrado de columnas está integrada en esta función.

    Args:
        ruta_base (str): Ruta al directorio principal de datos.
        subcarpetas_fuentes (list): Lista de nombres de subcarpetas (fuentes como 'difusas').
        lista_años (list): Lista de años (como strings) a procesar.
        mapa_contaminantes_est (dict): Diccionario para estandarizar nombres de contaminantes.
        mapa_regiones_est (dict): Diccionario para estandarizar nombres de regiones.

    Returns:
        pandas.DataFrame or None: Un DataFrame unificado con datos estandarizados,
                                  o None si ocurren errores críticos.
    """
    todos_los_datos = []
    # Definir los nombres de columna estándar en español
    columnas_estandar = ['año', 'region', 'comuna', 'cantidad_toneladas', 'contaminante', 'origen_carpeta']
    cols_esenciales = ['region', 'comuna', 'cantidad_toneladas', 'contaminante'] # Columnas que deben existir después de renombrar

    for año_actual in lista_años:
        for carpeta in subcarpetas_fuentes:
            ruta_carpeta = os.path.join(ruta_base, carpeta)
            if not os.path.isdir(ruta_carpeta):
                print(f"    Advertencia: Carpeta no encontrada: '{ruta_carpeta}'. Saltando.")
                continue
            archivo_encontrado = None
            try:
                archivos_potenciales = [f for f in os.listdir(ruta_carpeta) if año_actual in f and f.lower().endswith(('.csv', '.xlsx', '.xls'))]
                if archivos_potenciales:
                    archivos_csv = [f for f in archivos_potenciales if f.lower().endswith('.csv')]
                    if archivos_csv:
                        archivo_encontrado = archivos_csv[0]
                    else:
                        archivo_encontrado = archivos_potenciales[0]
            except FileNotFoundError:
                print(f"    Error: No se pudo acceder a la carpeta '{ruta_carpeta}'.")
                continue
            except Exception as e_listar:
                 print(f"    Error al listar archivos en '{ruta_carpeta}': {e_listar}")
                 continue

            if not archivo_encontrado:
                print(f"    INFO: No se encontró archivo para año {año_actual} en '{carpeta}'.")
                continue

            ruta_archivo = os.path.join(ruta_carpeta, archivo_encontrado)
            df_temporal = None
            print(f"    Leyendo archivo: '{archivo_encontrado}'")

            try:
                if ruta_archivo.lower().endswith('.csv'):
                    try:
                       df_temporal = pd.read_csv(ruta_archivo, encoding='utf-8', sep=';', decimal=',', on_bad_lines='warn', low_memory=False)
                    except UnicodeDecodeError:
                       df_temporal = pd.read_csv(ruta_archivo, encoding='latin-1', sep=';', decimal=',', on_bad_lines='warn', low_memory=False)
                    except Exception as error_csv:
                         print(f"      ERROR leyendo CSV: {type(error_csv).__name__} - {error_csv}")
                         continue # Saltar archivo
                elif ruta_archivo.lower().endswith(('.xlsx', '.xls')):
                     df_temporal = pd.read_excel(ruta_archivo, engine='openpyxl')
            except Exception as error_lectura:
                 print(f"      ERROR: Falló la lectura del archivo. Error: {type(error_lectura).__name__} - {error_lectura}")
                 continue # Saltar archivo

            if df_temporal is not None and not df_temporal.empty:
                df_temporal.columns = df_temporal.columns.str.strip()

                df_temporal['año'] = año_actual
                df_temporal['origen_carpeta'] = carpeta # Añadir información de la fuente

                dicc_renombrar = {}
                if 'REGION' in df_temporal.columns: dicc_renombrar['REGION'] = 'region'
                elif 'nom_region' in df_temporal.columns: dicc_renombrar['nom_region'] = 'region'
                # Comuna
                if 'NOM_COM' in df_temporal.columns: dicc_renombrar['NOM_COM'] = 'comuna'
                elif 'Comuna_Ruta' in df_temporal.columns: dicc_renombrar['Comuna_Ruta'] = 'comuna'
                # Contaminante
                if 'codigo_parametro' in df_temporal.columns: dicc_renombrar['codigo_parametro'] = 'contaminante'
                elif 'codigo_parameter' in df_temporal.columns: dicc_renombrar['codigo_parameter'] = 'contaminante'
                elif 'codigo_contaminante_ruta' in df_temporal.columns: dicc_renombrar['codigo_contaminante_ruta'] = 'contaminante'
                elif 'contaminantes' in df_temporal.columns: dicc_renombrar['contaminantes'] = 'contaminante' # Variación común
                # Valor (asumiendo 'cantidad_toneladas' como el más común y ya estándar)
                if 'cantidad_toneladas' not in df_temporal.columns:
                     print(f"      Advertencia: Columna 'cantidad_toneladas' no encontrada en {archivo_encontrado}. Verifique el archivo.")
                if dicc_renombrar:
                    df_temporal.rename(columns=dicc_renombrar, inplace=True)
                columnas_faltantes = [col for col in cols_esenciales if col not in df_temporal.columns]

                if columnas_faltantes:
                    print(f"      ¡ERROR GRAVE! Faltan columnas estándar después del intento de renombrado: {columnas_faltantes}. Archivo: {archivo_encontrado}. Saltando.")
                    continue
                try:
                    columnas_std_faltantes = [sc for sc in columnas_estandar if sc not in df_temporal.columns]
                    if columnas_std_faltantes:
                         raise KeyError(f"Columnas estándar faltantes antes de la selección final: {columnas_std_faltantes}")
                    df_temporal = df_temporal[columnas_estandar]
                except KeyError as e:
                    print(f"      ¡ERROR GRAVE! No se pudieron seleccionar las columnas estándar finales. {e}. Archivo: {archivo_encontrado}. Saltando.")
                    continue

                todos_los_datos.append(df_temporal)

            elif df_temporal is not None and df_temporal.empty:
                print(f"      INFO: Archivo '{archivo_encontrado}' leído pero resultó vacío.")

    if not todos_los_datos:
        print("\nError Crítico: No se cargaron datos válidos de ninguna fuente.")
        return None

    print(f"\nUnificando datos de {len(todos_los_datos)} archivo(s)...")
    try:
        df_unificado = pd.concat(todos_los_datos, ignore_index=True)
    except Exception as error_concat:
        print(f"Error Crítico al concatenar DataFrames: {error_concat}")
        return None
    print(f"DataFrame unificado creado con {len(df_unificado)} filas iniciales.")

    df_unificado['año'] = df_unificado['año'].astype(str)
    df_unificado['origen_carpeta'] = df_unificado['origen_carpeta'].astype(str)

    try:
        df_unificado['cantidad_toneladas'] = pd.to_numeric(df_unificado['cantidad_toneladas'], errors='coerce')
    except Exception as e_numeric:
        print(f"Error inesperado al convertir 'cantidad_toneladas' a numérico: {e_numeric}")
        return None

    columnas_para_dropna = ['año', 'region', 'comuna', 'cantidad_toneladas', 'contaminante']
    filas_antes_nan = len(df_unificado)
    df_unificado.dropna(subset=columnas_para_dropna, inplace=True)
    filas_despues_nan = len(df_unificado)
    print(f"Limpieza: Se eliminaron {filas_antes_nan - filas_despues_nan} filas con valores nulos en columnas clave.")

    if df_unificado.empty:
        print("Error: No quedan datos válidos después de la limpieza de NaNs.")
        return None
        
    print("Estandarizando nombres de Región, Comuna y Contaminante...")
    try:
        df_unificado['region'] = df_unificado['region'].astype(str).str.strip()
        df_unificado['region'] = df_unificado['region'].replace(mapa_regiones_est)
        df_unificado['comuna'] = df_unificado['comuna'].astype(str).str.strip().str.title()
        df_unificado['contaminante'] = df_unificado['contaminante'].astype(str).str.strip()
        df_unificado['contaminante'] = df_unificado['contaminante'].apply(lambda x: mapa_contaminantes_est.get(x, x))
        print("  Estandarización completada.")
    except Exception as e_std:
        print(f"Error inesperado durante la estandarización: {e_std}")
        return None

    print(f"--- Preparación de Datos Finalizada ---")
    print(f"DataFrame final listo con {len(df_unificado)} filas válidas.")
    return df_unificado

In [6]:
# Funciones para graficar

In [7]:
# %%
def generar_grafico_contaminante_especifico(df, contaminante_a_filtrar, directorio_salida, año_grafico):
    try:
        contaminante_mayuscula = contaminante_a_filtrar.upper()
        df_filtrado = df[df['contaminante'].astype(str).str.upper() == contaminante_mayuscula].copy()

        if df_filtrado.empty:
            print(f"  INFO ({año_grafico}): No se encontraron datos para '{contaminante_a_filtrar}'.")
            return

        datos_agrupados = df_filtrado.groupby(['region', 'origen_carpeta'])['cantidad_toneladas'].sum()
        df_grafico = datos_agrupados.unstack(level='origen_carpeta', fill_value=0)
        df_grafico = df_grafico.loc[df_grafico.sum(axis=1) > 1e-6].sort_index()

        if df_grafico.empty:
            print(f"  INFO ({año_grafico}): No hay emisiones significativas de '{contaminante_a_filtrar}' para graficar.")
            return

        ax = df_grafico.plot(kind='bar', stacked=False, figsize=(18, 8), width=0.8, colormap='tab10')
        plt.title(f'Emisiones de {contaminante_a_filtrar} por Región y Fuente ({año_grafico})', fontsize=16)
        plt.xlabel('Región', fontsize=12)
        plt.ylabel('Suma Total (Toneladas)', fontsize=12) # Etiqueta genérica
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(title='Tipo de Fuente')
        plt.tight_layout()

        nombre_cont_limpio = "".join(c if c.isalnum() else "_" for c in contaminante_a_filtrar)
        nombre_archivo = f'grafico_ABS_{nombre_cont_limpio}_region_fuente_{año_grafico}.png'
        ruta_guardado = os.path.join(directorio_salida, nombre_archivo)
        plt.savefig(ruta_guardado, dpi=100)
        plt.close()

    except KeyError as ke:
        print(f"  ¡ERROR de Clave ({año_grafico})! Graf. Absoluto '{contaminante_a_filtrar}'. Columna faltante: {ke}. Asegúrese que las columnas estándar existen.")
    except Exception as e_plot:
        print(f"  ¡ERROR Inesperado ({año_grafico})! Graf. Absoluto '{contaminante_a_filtrar}': {type(e_plot).__name__} - {e_plot}")

In [8]:
def grafico_proporcion(df, contaminante_a_filtrar, directorio_salida, año_grafico):
    try:
        contaminante_mayuscula = contaminante_a_filtrar.upper()
        # Filtrar usando columna estándar 'contaminante'
        df_filtrado = df[df['contaminante'].astype(str).str.upper() == contaminante_mayuscula].copy()

        if df_filtrado.empty:
            print(f"  INFO ({año_grafico}): No se encontraron datos para '{contaminante_a_filtrar}'.")
            return

        datos_agrupados = df_filtrado.groupby(['region', 'origen_carpeta'])['cantidad_toneladas'].sum()
        datos_agrupados = datos_agrupados[datos_agrupados > 1e-9]

        if datos_agrupados.empty:
             print(f"  INFO ({año_grafico}): No hay emisiones positivas para calcular proporciones de '{contaminante_a_filtrar}'.")
             return

        datos_porcentaje = datos_agrupados.groupby(level='region', group_keys=False).apply(lambda x: 100 * x / float(x.sum()))
        df_grafico = datos_porcentaje.unstack(level='origen_carpeta', fill_value=0)
        df_grafico = df_grafico.loc[df_grafico.sum(axis=1) > 1e-6].sort_index()

        if df_grafico.empty:
            print(f"  INFO ({año_grafico}): No hay datos significativos para proporciones de '{contaminante_a_filtrar}'.")
            return

        
        ax = df_grafico.plot(kind='bar', stacked=True, figsize=(18, 8), width=0.8, colormap='tab10')
        plt.title(f'Proporción de Fuentes de {contaminante_a_filtrar} por Región ({año_grafico})', fontsize=16)
        plt.xlabel('Región', fontsize=12)
        plt.ylabel('Porcentaje (%)', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(np.arange(0, 101, 10), fontsize=10)
        plt.ylim(0, 100)
        plt.axhline(50, color='grey', linestyle='--', linewidth=0.8)
        plt.legend(title='Tipo de Fuente', bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout(rect=[0, 0, 0.9, 1]) # Ajustar layout para leyenda

        nombre_cont_limpio = "".join(c if c.isalnum() else "_" for c in contaminante_a_filtrar)
        nombre_archivo = f'grafico_PROP_{nombre_cont_limpio}_region_fuente_{año_grafico}.png'
        ruta_guardado = os.path.join(directorio_salida, nombre_archivo)
        plt.savefig(ruta_guardado, bbox_inches='tight', dpi=100) # Usar bbox_inches='tight'
        plt.close()

    except KeyError as ke:
         print(f"  ¡ERROR de Clave ({año_grafico})! Graf. Proporción '{contaminante_a_filtrar}'. Columna faltante: {ke}. Asegúrese que las columnas estándar existen.")
    except ZeroDivisionError:
         print(f"  ¡ERROR ({año_grafico})! División por cero al calcular % para '{contaminante_a_filtrar}'. Revise si hay regiones con suma total cero.")
    except Exception as e_plot:
         print(f"  ¡ERROR Inesperado ({año_grafico})! Graf. Proporción '{contaminante_a_filtrar}': {type(e_plot).__name__} - {e_plot}")

In [9]:
# %%
def generar_grafico_contaminantes_ciudad(df, nombre_ciudad, top_n, directorio_salida, año_grafico):
    """Genera gráfico de barras con Top N contaminantes para una ciudad/comuna y AÑO."""
    try:
        ciudad_a_buscar = nombre_ciudad.title()
        df_ciudad = df[df['comuna'].astype(str) == ciudad_a_buscar].copy()

        if df_ciudad.empty:
            print(f"  INFO ({año_grafico}): No se encontraron datos para '{nombre_ciudad}' (buscada como '{ciudad_a_buscar}').")
            return

        df_ciudad['cantidad_toneladas'] = pd.to_numeric(df_ciudad['cantidad_toneladas'], errors='coerce')
        df_ciudad.dropna(subset=['cantidad_toneladas', 'contaminante'], inplace=True)

        if df_ciudad.empty:
            print(f"  INFO ({año_grafico}): No quedan datos numéricos válidos para '{nombre_ciudad}' después de limpiar NaNs.")
            return

        suma_cont_ciudad = df_ciudad.groupby('contaminante')['cantidad_toneladas'].sum()
        suma_cont_ciudad = suma_cont_ciudad[suma_cont_ciudad > 1e-9]

        if suma_cont_ciudad.empty:
            print(f"  INFO ({año_grafico}): No hay emisiones positivas registradas para contaminantes en '{nombre_ciudad}'.")
            return

        n_real = min(top_n, len(suma_cont_ciudad))
        if n_real == 0:
             print(f"  INFO ({año_grafico}): No hay datos de contaminantes > 0 para graficar.")
             return
        suma_top_ciudad = suma_cont_ciudad.nlargest(n_real)

        plt.figure(figsize=(12, 7))
        colores = sns.color_palette("viridis", n_colors=n_real)
        suma_top_ciudad.sort_values(ascending=False).plot(kind='bar', color=colores)
        plt.title(f'Top {n_real} Contaminantes Emitidos en {nombre_ciudad} ({año_grafico})', fontsize=15)
        plt.xlabel('Contaminante', fontsize=12)
        plt.ylabel('Emisiones Totales (toneladas)', fontsize=12) # Asumiendo unidad
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.tight_layout() # Ajustar layout

        nombre_ciudad_limpio = "".join(c if c.isalnum() else "_" for c in nombre_ciudad)
        nombre_archivo = f'grafico_top{n_real}_contaminantes_{nombre_ciudad_limpio}_{año_grafico}.png'
        ruta_guardado = os.path.join(directorio_salida, nombre_archivo)
        plt.savefig(ruta_guardado, dpi=100)
        plt.close()

    except KeyError as ke:
        print(f"  ¡ERROR de Clave ({año_grafico})! Graf. Ciudad '{nombre_ciudad}'. Columna faltante: {ke}. Asegúrese que las columnas 'comuna', 'cantidad_toneladas', 'contaminante' existen.")
    except Exception as e_plot:
        print(f"  ¡ERROR Inesperado ({año_grafico})! Graf. Ciudad '{nombre_ciudad}': {type(e_plot).__name__} - {e_plot}")

In [10]:
datos_unificados = None
try:
    os.makedirs(CARPETA_GRAFICOS, exist_ok=True)
    print(f"Directorio de gráficos: '{CARPETA_GRAFICOS}'")
except OSError as e:
    print(f"Error Fatal: No se pudo crear el directorio de gráficos '{CARPETA_GRAFICOS}'. Error: {e}")
else:
    datos_unificados = cargar_y_preparar_datos(
        RUTA_BASE_DATOS, SUBCARPETAS, AÑOS_A_PROCESAR,
        MAPA_CONTAMINANTES,
        MAPEO_ESTANDARIZACION_REGION
    )

# --- 2. Generar Gráficos (si la carga de datos fue exitosa) ---
if datos_unificados is not None and not datos_unificados.empty:
    años_en_datos = sorted(datos_unificados['año'].unique())
    for año_actual_grafico in años_en_datos:
        df_año = datos_unificados[datos_unificados['año'] == año_actual_grafico]
        if df_año.empty:
            continue

        generar_grafico_contaminante_especifico(df_año, NOMBRE_MP25, CARPETA_GRAFICOS, año_actual_grafico)
        grafico_proporcion(df_año, NOMBRE_MP25, CARPETA_GRAFICOS, año_actual_grafico)
        generar_grafico_contaminante_especifico(df_año, NOMBRE_MP10, CARPETA_GRAFICOS, año_actual_grafico)
        grafico_proporcion(df_año, NOMBRE_MP10, CARPETA_GRAFICOS, año_actual_grafico)
        generar_grafico_contaminantes_ciudad(df_año,NOMBRE_CIUDAD_FILTRO,TOP_N_CONTAMINANTES_CIUDAD,CARPETA_GRAFICOS,año_actual_grafico)
        
elif datos_unificados is None:
     print("No hay df unificado")

# --- Mensaje Final ---
print(f"\n--- FIN: Ejecución del Análisis Multi-Año ({', '.join(AÑOS_A_PROCESAR)}) ---")
# Verificar si el directorio existe y contiene gráficos
if os.path.exists(CARPETA_GRAFICOS):
    try:
        if any(fname.endswith('.png') for fname in os.listdir(CARPETA_GRAFICOS)):
            print(f"Revisa los gráficos generados en la carpeta: '{CARPETA_GRAFICOS}'")
        else:
             print(f"Se creó la carpeta '{CARPETA_GRAFICOS}', pero no contiene archivos PNG generados.")
    except Exception as e_list_final:
        print(f"No se pudo verificar el contenido de la carpeta de gráficos: {e_list_final}")
else:
     print(f"No se generaron gráficos (posiblemente debido a errores previos o directorio no creado).")

Directorio de gráficos: 'graficos_2019-2023_reporte_es'
    Leyendo archivo: 'ruea-efd-2019-ckan.csv'
    Leyendo archivo: 'ruea-efp-2019-ckan.csv'
    Leyendo archivo: 'ruea-tr-2019-ckan.csv'
    Leyendo archivo: 'ruea-efd-2020-ckan.csv'
    Leyendo archivo: 'ruea-efp-2020-ckan.csv'
    Leyendo archivo: 'ruea-tr-2020-ckan.csv'
    Leyendo archivo: 'ruea-efd-2021-ckan.csv'
    Leyendo archivo: 'ruea-efp-2021-ckan.xlsx'
    Leyendo archivo: 'ruea-tr-2021-ckan.csv'
    Leyendo archivo: 'ruea-efd-2022-ckan.csv'
    Leyendo archivo: 'ruea-efp-2022-ckan.csv'
    Leyendo archivo: 'ruea-tr-2022-ckan.csv'
    Leyendo archivo: 'ruea-efd-2023-ckan.csv'
    Leyendo archivo: 'ruea-efp-2023-ckan.csv'
    Leyendo archivo: 'ruea-tr-2023-ckan.csv'

Unificando datos de 15 archivo(s)...
DataFrame unificado creado con 5398387 filas iniciales.
Limpieza: Se eliminaron 175339 filas con valores nulos en columnas clave.
Estandarizando nombres de Región, Comuna y Contaminante...
  Estandarización completada.
-

In [11]:
# %% [markdown]
# ## Revision de daros

In [12]:
if 'datos_unificados' in locals() and datos_unificados is not None and not datos_unificados.empty:
    print(f"\nForma del DataFrame (filas, columnas): {datos_unificados.shape}")
    print("\nPrimeras 5 filas:")
    from IPython.display import display
    display(datos_unificados.head())

    print("\nInformación General (Tipos de datos, Nulos):")
    datos_unificados.info()

    if 'cantidad_toneladas' in datos_unificados.columns and pd.api.types.is_numeric_dtype(datos_unificados['cantidad_toneladas']):
        print(f"\nEstadísticas descriptivas para 'cantidad_toneladas':")
        display(datos_unificados['cantidad_toneladas'].describe().apply("{:,.2f}".format))
        # Check for negative values which might indicate data issues
        if (datos_unificados['cantidad_toneladas'] < 0).any():
             neg_count = (datos_unificados['cantidad_toneladas'] < 0).sum()
             print(f"  ADVERTENCIA: Se encontraron {neg_count} registros con valores negativos en 'cantidad_toneladas'.")
    else:
        print("\nColumna 'cantidad_toneladas' no es numérica o no existe, no se muestran estadísticas descriptivas.")

    print("\nValores Únicos en columnas clave:")
    columnas_clave_preview = ['region', 'contaminante', 'origen_carpeta']
    for col in columnas_clave_preview:
        if col in datos_unificados.columns:
            try:
                num_unicos = datos_unificados[col].nunique()
                print(f"- Columna '{col}': {num_unicos} valores únicos.")
                all_uniques = sorted(datos_unificados[col].unique())
                if num_unicos < 25:
                    print(f"  Valores: {all_uniques}")
                else: # Show a sample
                    print(f"  Muestra: {all_uniques[:10]}... (y {num_unicos-10} más)")
            except Exception as e_unique:
                 print(f"  Error al obtener únicos para '{col}': {e_unique}")
        else:
             print(f"- Advertencia: No se encontró la columna '{col}'.")

    # Specific check for the target city filter ('comuna')
    if 'comuna' in datos_unificados.columns:
        ciudad_filtrar_title = NOMBRE_CIUDAD_FILTRO.title()
        try:
            comunas_unicas = datos_unificados['comuna'].unique()
            ciudad_presente = ciudad_filtrar_title in comunas_unicas
            print(f"\n¿Está la ciudad '{NOMBRE_CIUDAD_FILTRO}' (buscada como '{ciudad_filtrar_title}') presente en 'comuna'?: {'Sí' if ciudad_presente else 'NO'}")
        except Exception as e_comuna:
             print(f"\nError al verificar la ciudad '{NOMBRE_CIUDAD_FILTRO}': {e_comuna}")


else:
    print("error")


Forma del DataFrame (filas, columnas): (5223048, 6)

Primeras 5 filas:


,año,region,comuna,cantidad_toneladas,contaminante,origen_carpeta
0,2019,Antofagasta,Antofagasta,3.037223,Carbono Negro,difusas
1,2019,Antofagasta,Mejillones,0.104612,Carbono Negro,difusas
2,2019,Antofagasta,Sierra Gorda,0.012500,Carbono Negro,difusas
3,2019,Antofagasta,Taltal,0.098901,Carbono Negro,difusas
4,2019,Antofagasta,Calama,1.361261,Carbono Negro,difusas



Información General (Tipos de datos, Nulos):
<class 'pandas.core.frame.DataFrame'>
Index: 5223048 entries, 0 to 5398386
Data columns (total 6 columns):
 #   Column              Dtype  
---  ------              -----  
 0   año                 object 
 1   region              object 
 2   comuna              object 
 3   cantidad_toneladas  float64
 4   contaminante        object 
 5   origen_carpeta      object 
dtypes: float64(1), object(5)
memory usage: 278.9+ MB

Estadísticas descriptivas para 'cantidad_toneladas':


count    5,223,048.00
mean           122.31
std         11,826.29
min           -518.24
25%              0.00
50%              0.01
75%              0.35
max      7,648,811.43
Name: cantidad_toneladas, dtype: object

  ADVERTENCIA: Se encontraron 178 registros con valores negativos en 'cantidad_toneladas'.

Valores Únicos en columnas clave:
- Columna 'region': 16 valores únicos.
  Valores: ['Antofagasta', 'Araucanía', 'Arica y Parinacota', 'Atacama', 'Aysén', 'Biobío', 'Coquimbo', 'Los Lagos', 'Los Ríos', 'Magallanes', 'Maule', 'Metropolitana', "O'Higgins", 'Tarapacá', 'Valparaíso', 'Ñuble']
- Columna 'contaminante': 20 valores únicos.
  Valores: ['Amoniaco (NH3)', 'Arsénico', 'Benceno', 'COV', 'Carbono Negro', 'Dibenzoparadioxinas policloradas y furanos (PCDD/F)', 'Dióxido de azufre (SO2)', 'Dióxido de carbono (CO2)', 'Hidrocarburos totales', 'MP10', 'MP2.5', 'Material Particulado Total', 'Mercurio', 'Metano (CH4)', 'Monóxido de carbono (CO)', 'NOx', 'Oxido Nitroso', 'Plomo', 'SOx', 'Tolueno']
- Columna 'origen_carpeta': 3 valores únicos.
  Valores: ['difusas', 'puntuales', 'ruta']

¿Está la ciudad 'Valdivia' (buscada como 'Valdivia') presente en 'comuna'?: Sí


In [13]:
# Analisis macrozonas

In [14]:
# Verificar si existen los datos
if 'datos_unificados' not in locals() or datos_unificados is None or datos_unificados.empty:
    print("Error: El DataFrame 'datos_unificados' no está disponible o está vacío. No se puede realizar el análisis.")
else:
    # --- 1. Definir y Aplicar Macrozonas ---
    print("1. Definiendo y aplicando Macrozonas...")
    # Usar nombres de región ya estandarizados
    mapa_regiones_macrozonas = {
        'Arica y Parinacota': 'Norte', 'Tarapacá': 'Norte', 'Antofagasta': 'Norte',
        'Atacama': 'Norte', 'Coquimbo': 'Norte',
        'Valparaíso': 'Centro', 'Metropolitana': 'Centro', "O'Higgins": 'Centro',
        'Maule': 'Centro', 'Ñuble': 'Centro', 'Biobío': 'Centro',
        'Araucanía': 'Sur', 'Los Ríos': 'Sur', 'Los Lagos': 'Sur',
        'Aysén': 'Austral',
        'Magallanes': 'Austral'
    }
    datos_analisis = datos_unificados.copy()
    datos_analisis['macrozona'] = datos_analisis['region'].map(mapa_regiones_macrozonas)
    regiones_no_mapeadas = datos_analisis[datos_analisis['macrozona'].isnull()]['region'].unique()
    if len(regiones_no_mapeadas) > 0:
        print(f"  Advertencia: Las siguientes regiones no fueron mapeadas a una macrozona: {list(regiones_no_mapeadas)}")
        datos_analisis.dropna(subset=['macrozona'], inplace=True)
        print(f"  Registros de {len(regiones_no_mapeadas)} regiones no mapeadas excluidos del análisis.")

    contaminantes_macrozona = [NOMBRE_MP25, NOMBRE_MP10, 'Dióxido de azufre (SO2)', 'Monóxido de carbono (CO)']
    print(f"\n2. Filtrando por contaminantes: {', '.join(contaminantes_macrozona)}")
    df_filtrado_cont = datos_analisis[datos_analisis['contaminante'].isin(contaminantes_macrozona)]

    if df_filtrado_cont.empty:
            print("  Error: No se encontraron datos para los contaminantes seleccionados en este análisis.")
    else:
        # --- 3. Calcular Emisiones Anuales por Macrozona/Contaminante ---
        print("\n3. Calculando emisiones anuales agrupadas por Macrozona y Contaminante...")
        emisiones_anuales = df_filtrado_cont.groupby(['macrozona', 'año', 'contaminante'])['cantidad_toneladas'].sum().reset_index()
        print(f"  Se calcularon {len(emisiones_anuales)} registros anuales agrupados.")

        if not emisiones_anuales.empty:
            # --- 4. Gráfico 1: Boxplot de Emisiones Anuales ---
            print("\n4. Generando Gráfico: Boxplot de Emisiones Anuales por Macrozona (Escala Log)...")
            plt.figure(figsize=(18, 9))
            eje_caja = sns.boxplot(data=emisiones_anuales,
                                 x='contaminante', y='cantidad_toneladas',
                                 hue='macrozona', order=contaminantes_macrozona,
                                 hue_order=['Norte', 'Centro', 'Sur', 'Austral'], palette='viridis', linewidth=1.5)
            plt.title('Distribución de Emisiones Anuales por Macrozona y Contaminante (2019-2023)', fontsize=18, pad=20)
            plt.xlabel('Contaminante', fontsize=14)
            plt.ylabel('Emisiones Totales Anuales (toneladas, escala log)', fontsize=14)
            plt.yscale('log')
            try:
                 eje_caja.yaxis.set_major_formatter(mticker.ScalarFormatter())
                 eje_caja.yaxis.get_major_formatter().set_scientific(False)
                 eje_caja.yaxis.get_major_formatter().set_useOffset(False)
            except Exception as error_formato:
                 print(f"   Advertencia: No se pudo aplicar formato especial al eje Y log: {error_formato}")
            plt.xticks(fontsize=12); plt.yticks(fontsize=12)
            plt.legend(title='Macrozona', fontsize=12, title_fontsize=13)
            plt.tight_layout()
            nombre_archivo_caja = f'grafico_boxplot_emisiones_macrozona_log_{AÑOS_A_PROCESAR[0]}-{AÑOS_A_PROCESAR[-1]}.png'
            ruta_guardado_caja = os.path.join(CARPETA_GRAFICOS, nombre_archivo_caja)
            try:
                plt.savefig(ruta_guardado_caja, dpi=120)
                print(f"  -> Gráfico Boxplot guardado en: '{ruta_guardado_caja}'")
            except Exception as e_guardar: print(f"  Error al guardar Boxplot: {e_guardar}")
            plt.close()
        else:
            print("  No hay datos de emisiones anuales agrupadas para generar el boxplot.")

        df_mp25 = datos_analisis[datos_analisis['contaminante'] == NOMBRE_MP25]
        if not df_mp25.empty:
            suma_fuente = df_mp25.groupby(['macrozona', 'año', 'origen_carpeta'])['cantidad_toneladas'].sum()
            total_año = suma_fuente.groupby(level=['macrozona', 'año']).transform('sum')
            porcentaje_fuente = (suma_fuente / total_año.replace(0, np.nan)) * 100
            porcentaje_fuente = porcentaje_fuente.dropna()
            if not porcentaje_fuente.empty:
                 pct_promedio_fuente = porcentaje_fuente.groupby(level=['macrozona', 'origen_carpeta']).mean().reset_index()
                 pct_promedio_fuente.rename(columns={'cantidad_toneladas': 'porcentaje_promedio'}, inplace=True)
                 plt.figure(figsize=(14, 8))
                 eje_barra = sns.barplot(data=pct_promedio_fuente, x='macrozona', y='porcentaje_promedio',
                                      hue='origen_carpeta', order=['Norte', 'Centro', 'Sur', 'Austral'],
                                      hue_order=['difusas', 'puntuales', 'ruta'], palette='Set2', edgecolor='black', linewidth=0.8)
                 plt.title(f'Contribución Promedio (%) de Fuentes a Emisiones {NOMBRE_MP25} por Macrozona (2019-2023)', fontsize=16, pad=20)
                 plt.xlabel('Macrozona', fontsize=14)
                 plt.ylabel('Contribución Promedio Anual (%)', fontsize=14)
                 plt.xticks(fontsize=12); plt.yticks(np.arange(0, 101, 10), fontsize=12)
                 plt.ylim(0, 100)
                 plt.legend(title='Tipo de Fuente', fontsize=12, title_fontsize=13)
                 plt.tight_layout()
                 nombre_archivo_barra = f'grafico_barras_pct_fuente_mp25_macrozona_{AÑOS_A_PROCESAR[0]}-{AÑOS_A_PROCESAR[-1]}.png'
                 ruta_guardado_barra = os.path.join(CARPETA_GRAFICOS, nombre_archivo_barra)
                 try:
                     plt.savefig(ruta_guardado_barra, dpi=120)
                     print(f"  -> Gráfico Barras % guardado en: '{ruta_guardado_barra}'")
                 except Exception as e_guardar: print(f"  Error al guardar Gráfico Barras %: {e_guardar}")
                 plt.close()
        else:
             print(f"  No se encontraron datos de {NOMBRE_MP25} para calcular la contribución de fuentes.")

--- Iniciando Análisis Descriptivo por Macrozonas ---
1. Definiendo y aplicando Macrozonas...

2. Filtrando por contaminantes: MP2.5, MP10, Dióxido de azufre (SO2), Monóxido de carbono (CO)

3. Calculando emisiones anuales agrupadas por Macrozona y Contaminante...
  Se calcularon 80 registros anuales agrupados.

4. Generando Gráfico: Boxplot de Emisiones Anuales por Macrozona (Escala Log)...
  -> Gráfico Boxplot guardado en: 'graficos_2019-2023_reporte_es/grafico_boxplot_emisiones_macrozona_log_2019-2023.png'
  Cálculo de porcentajes promedio completado.

6. Generando Gráfico: Barras de Contribución % Promedio Fuentes (MP2.5)...
  -> Gráfico Barras % guardado en: 'graficos_2019-2023_reporte_es/grafico_barras_pct_fuente_mp25_macrozona_2019-2023.png'


In [15]:
# %%
print(f"--- Generando Gráficos de Tendencia para {NOMBRE_MP25} (Separados por Fuente) ---")

if 'datos_unificados' not in locals() or datos_unificados is None or datos_unificados.empty:
    print("Error: El DataFrame 'datos_unificados' no está disponible o está vacío. No se puede generar este gráfico.")
else:
    # --- Preparación ---
    lista_años_ordenada = sorted(datos_unificados['año'].unique())
    fuentes_a_graficar = sorted(datos_unificados['origen_carpeta'].unique())

    # Determinar orden de regiones basado en emisión total de MP2.5
    try:
        orden_regiones = datos_unificados[datos_unificados['contaminante'] == NOMBRE_MP25]\
                             .groupby('region')['cantidad_toneladas'].sum()\
                             .sort_values(ascending=False).index.tolist()
        print(f"Orden de regiones determinado por emisión total de {NOMBRE_MP25}.")
    except KeyError:
        print(f"Advertencia: No se pudo calcular el orden de regiones. Usando orden alfabético.")
        orden_regiones = sorted(datos_unificados['region'].unique())

    print(f"Fuentes a graficar: {fuentes_a_graficar}")

    # --- Bucle para cada tipo de fuente ---
    for fuente_actual in fuentes_a_graficar:
        print(f"\n-- Generando gráfico para Fuente: '{fuente_actual}' | Contaminante: {NOMBRE_MP25} --")

        # Filtrar datos para la fuente actual Y MP2.5
        df_fuente_mp25 = datos_unificados[
            (datos_unificados['origen_carpeta'] == fuente_actual) &
            (datos_unificados['contaminante'] == NOMBRE_MP25)
        ]

        if not df_fuente_mp25.empty:
            datos_grafico = df_fuente_mp25.groupby(['region', 'año'])['cantidad_toneladas'].sum().reset_index()

            if datos_grafico.empty:
                print(f"  INFO: No hay datos agregados para graficar para la fuente '{fuente_actual}'.")
                continue

            plt.figure(figsize=(18, 8))
            ax = sns.barplot(data=datos_grafico,
                             x='region', y='cantidad_toneladas', hue='año',
                             order=orden_regiones,
                             hue_order=lista_años_ordenada,
                             palette='viridis'
                            )

            plt.title(f'Emisiones Totales de {NOMBRE_MP25} (Fuente: {fuente_actual.capitalize()}) por Región y Año', fontsize=16, pad=15)
            plt.xlabel('Región', fontsize=13)
            plt.ylabel('Emisiones Totales (toneladas)', fontsize=13)
            plt.xticks(rotation=45, ha='right', fontsize=11)
            plt.yticks(fontsize=11)
            plt.legend(title='Año', title_fontsize='12', fontsize='11')
            plt.tight_layout()

            # Guardar gráfico
            nombre_archivo = f'grafico_tendencia_{NOMBRE_MP25}_region_año_FUENTE_{fuente_actual}_{AÑOS_A_PROCESAR[0]}-{AÑOS_A_PROCESAR[-1]}.png'
            ruta_guardado = os.path.join(CARPETA_GRAFICOS, nombre_archivo)
            try:
                plt.savefig(ruta_guardado, dpi=120)
                print(f"  -> Gráfico guardado en: '{ruta_guardado}'")
            except Exception as e_guardar:
                print(f"  Error al guardar gráfico para '{fuente_actual}': {e_guardar}")
            plt.close() # Cerrar figura

        else:
            print(f"  INFO: No se encontraron datos de {NOMBRE_MP25} para la fuente '{fuente_actual}'.")

--- Generando Gráficos de Tendencia para MP2.5 (Separados por Fuente) ---
Orden de regiones determinado por emisión total de MP2.5.
Fuentes a graficar: ['difusas', 'puntuales', 'ruta']

-- Generando gráfico para Fuente: 'difusas' | Contaminante: MP2.5 --
  -> Gráfico guardado en: 'graficos_2019-2023_reporte_es/grafico_tendencia_MP2.5_region_año_FUENTE_difusas_2019-2023.png'

-- Generando gráfico para Fuente: 'puntuales' | Contaminante: MP2.5 --
  -> Gráfico guardado en: 'graficos_2019-2023_reporte_es/grafico_tendencia_MP2.5_region_año_FUENTE_puntuales_2019-2023.png'

-- Generando gráfico para Fuente: 'ruta' | Contaminante: MP2.5 --
  -> Gráfico guardado en: 'graficos_2019-2023_reporte_es/grafico_tendencia_MP2.5_region_año_FUENTE_ruta_2019-2023.png'

--- Fin de la Generación de Gráficos de Tendencia por Fuente ---
